# Weather Classification Project

## Overview
This project builds a machine learning classifier to predict weather conditions based on hourly meteorological data from weather stations across the U.S. The classifier categorizes weather into five classes: **Clear**, **Cloudy**, **Rain**, **Snow**, and **Storm (Not sampled due to low label rate)**.

## Data Pipeline
1. **Data Collection**: Retrieved 2024 hourly weather data from 20 weather stations using the [Meteostat API](https://dev.meteostat.net/)
2. **Feature Engineering**: 
   - Dropped Highly missing features (total sunshine, snow depth, precipitation, wind gust)
   - Created lagged features (lag 1, 2, 3) for all meteorological variables to capture temporal patterns
3. **Data Cleaning**: Removed rows with missing values before sampling
4. **Geographic Integration**: Merged latitude/longitude data to enable spatial analysis

## Target Variable
**Weather Condition (coco)** — Grouped into 5 categories:
- **Clear**
- **Cloudy**
- **Rain**
- **Snow**
- **Storm** 

## Model 
- **Algorithm**: Random Forest Classifier
- **Train-Test Split**: 80/20 with stratification

## Performance metrics 
- **Accuracy**:  0.7279
- **Precision**: 0.7258
- **Recall**:    0.7279
- **F1 Score**:  0.7156


In [11]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from meteostat import Point, Stations, Hourly, Daily

from great_tables import GT, md, style, loc
import altair as alt
from vega_datasets import data as datasets

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [12]:
stations = Stations()
stations = stations.nearby(40.4406, -79.9959)
stations = stations.fetch()

# Sample 50 US stations and split into train/test cities
all_stations = stations[stations["country"]=="US"].sample(50, random_state=42)

# Split cities 80/20 (40 train, 10 test)
train_stations = all_stations.sample(40, random_state=42)
test_stations = all_stations.drop(train_stations.index)

# Use all stations for initial map
stations = all_stations

# --- map base ---
us_states = alt.topo_feature(datasets.us_10m.url, 'states')

base = (
    alt.Chart(us_states)
    .mark_geoshape(fill="#f0f0f0", stroke="black")
    .project("albersUsa")
)

# --- stations ---
points = (
    alt.Chart(stations.reset_index())
    .mark_circle(size=200, opacity=0.8)
    .encode(
        longitude="longitude:Q",
        latitude= "latitude:Q",
        color="name:N",
        tooltip=["name:N","id:N" , "longitude:Q", "latitude:Q"]
    )
)
(base + points ).properties(
    width=600,
    height=400,
    title="50 Weather Stations (40 Train + 10 Test)"
)


Train cities: 40
Test cities: 10


alt.LayerChart(...)

In [ ]:

# Collect data for all stations (2024)
data = Hourly(stations, datetime.datetime(2024, 1, 1), datetime.datetime(2024, 12, 31))
data = data.fetch()

data = data.drop(columns=["tsun", "snow", "prcp","wpgt"])


# 1) Ensure time level is datetime
data = data.copy()
data.index = data.index.set_levels(
    pd.to_datetime(data.index.levels[1]),
    level=1
)

# 2) Sort by station -> time
data = data.sort_index(level=["station", "time"])

# 2.5) Keep wind direction as regular numeric feature (0-360 degrees)
data = data.sort_index()
data['pres_change'] = data.groupby(level='station')['pres'].diff()
data['rhum_change'] = data.groupby(level='station')['rhum'].diff()
data['wspd_change'] = data.groupby(level='station')['wspd'].diff()

# Weather interaction features (combinations that indicate specific conditions)
data['pres_rhum_ratio'] = data['pres'] / (data['rhum'] + 0.01)  # Dry vs humid pressure
data['wspd_rhum_product'] = data['wspd'] * data['rhum']  # Wind-moisture interaction
data['pres_wspd_ratio'] = data['pres'] / (data['wspd'] + 0.01)  # Pressure stability vs wind
# Weather interaction features (combinations that indicate specific conditions)
data['pres_rhum_ratio'] = data['pres'] / (data['rhum'] + 0.01)
data['wspd_rhum_product'] = data['wspd'] * data['rhum']
data['pres_wspd_ratio'] = data['pres'] / (data['wspd'] + 0.01)

# Atmospheric stability indicators
data['rhum_squared'] = data['rhum'] ** 2
data['wspd_squared'] = data['wspd'] ** 2
data['pres_deviation'] = np.abs(data['pres'] - data['pres'].mean())

# Weather extremeness indicators (handle NaN values)
data['rhum_extreme'] = np.where(data['rhum'].fillna(0) > 90, 1, 0)
data['low_pressure'] = np.where(data['pres'].fillna(data['pres'].mean()) < data['pres'].quantile(0.1), 1, 0)
data['high_wind'] = np.where(data['wspd'].fillna(0) > data['wspd'].quantile(0.9), 1, 0)

# Temporal weather patterns (using existing data before lags)
data['pressure_trend'] = data.groupby(level='station')['pres'].rolling(3, min_periods=1).mean().reset_index(0, drop=True)
data['humidity_volatility'] = data.groupby(level='station')['rhum'].rolling(3, min_periods=1).std().reset_index(0, drop=True).fillna(0)
data['wind_consistency'] = data.groupby(level='station')['wspd'].rolling(3, min_periods=1).std().reset_index(0, drop=True).fillna(0)

data['temp_change'] = data.groupby(level='station')['temp'].diff()
data['dwpt_change'] = data.groupby(level='station')['dwpt'].diff()

# Temperature-based weather indicators
data['temp_dwpt_spread'] = data['temp'] - data['dwpt']  # Temperature-dew point spread (humidity indicator)
data['temp_dwpt_spread'] = data['temp'] - data['dwpt']
data['heat_index'] = data['temp'] + 0.5 * (data['rhum'] / 100) * (data['temp'] - 14.5)
data['temp_extreme_hot'] = np.where(data['temp'].fillna(data['temp'].mean()) > data['temp'].quantile(0.9), 1, 0)
data['temp_extreme_cold'] = np.where(data['temp'].fillna(data['temp'].mean()) < data['temp'].quantile(0.1), 1, 0)

# Temperature interactions with other variables
data['temp_pres_ratio'] = data['temp'] / (data['pres'] / 1000 + 0.01)
data['temp_wspd_product'] = data['temp'] * data['wspd']
data['dwpt_rhum_ratio'] = data['dwpt'] / (data['rhum'] + 0.01)

# Advanced temperature indicators
data['temp_volatility'] = data.groupby(level='station')['temp'].rolling(3, min_periods=1).std().reset_index(0, drop=True).fillna(0)
data['dwpt_trend'] = data.groupby(level='station')['dwpt'].rolling(3, min_periods=1).mean().reset_index(0, drop=True)
data['temp_trend'] = data.groupby(level='station')['temp'].rolling(3, min_periods=1).mean().reset_index(0, drop=True)

# Weather comfort and stability indicators (handle NaN values carefully)
data['comfort_index'] = 0
data['frost_risk'] = 0
data['condensation_risk'] = 0
temp_valid = data['temp'].notna()
dwpt_valid = data['dwpt'].notna() 
rhum_valid = data['rhum'].notna()

# Comfort index: moderate temperature and humidity
comfort_mask = temp_valid & rhum_valid & (data['temp'] >= 18) & (data['temp'] <= 24) & (data['rhum'] >= 30) & (data['rhum'] <= 70)
data.loc[comfort_mask, 'comfort_index'] = 1
comfort_mask = temp_valid & rhum_valid & (data['temp'] >= 18) & (data['temp'] <= 24) & (data['rhum'] >= 30) & (data['rhum'] <= 70)
data.loc[comfort_mask, 'comfort_index'] = 1

frost_mask = temp_valid & dwpt_valid & (data['temp'] <= 2) & (data['dwpt'] <= 0)
data.loc[frost_mask, 'frost_risk'] = 1

condensation_mask = temp_valid & dwpt_valid & (np.abs(data['temp'] - data['dwpt']) <= 2)
data.loc[condensation_mask, 'condensation_risk'] = 1

# 3) Identify columns to lag - RESTORED for improved model performance
cols = list(data.columns)
cols.remove("coco")  
cols = list(data.columns)
cols.remove("coco")
for lag in [1, 2, 3]:
    lagged = data.groupby(level="station")[cols].shift(lag)
    lagged.columns = [f"{c}_lag{lag}" for c in cols]
    data = data.join(lagged)

# Remove rows with missing values after lag feature creation
data = data.dropna()
# Remove rows with missing values after lag feature creation
data = data.dropna()
data_reset = data.reset_index()
data_with_coords = data_reset.merge(
    stations[['latitude', 'longitude', 'elevation']],
    left_on='station',
    right_index=True,
    how='left'
)

data = data_with_coords.set_index(['station', 'time'])

print(f"Data length after lag features and cleanup: {len(data)}")
data = data[~data['coco'].isin([23, 24, 25, 26, 27])]
# Remove storm condition codes (23-27) from dataset
def group_condition_codes(code):
    if code in [1, 2]:
        return "Clear"
    elif code in [3, 4, 5, 6]:
        return "Cloudy"
    elif code in [7, 8, 9, 10, 11, 12, 13, 17, 18, 19, 20]:
        return "Rain"
    elif code in [14, 15, 16, 21, 22]:
        return "Snow"
    # Storm codes (23-27) have been removed from dataset

data["coco"] = data['coco'].apply(group_condition_codes)
data["coco"].value_counts(normalize=True) * 100

417959
Data length after lag features and cleanup: 408338
Data length after lag features and cleanup: 408338
Data length after removing storm conditions: 408316
Data length after removing storm conditions: 408316
Data length after removing storm conditions: 408316
Data length after removing storm conditions: 408316


In [14]:
# -----------------------------
# Implement Additional Features from Multiple Categories
# -----------------------------

# First, let's add features we can calculate from existing data and time information
enhanced_data = data.copy()
original_features = len(enhanced_data.columns) - 1  # -1 for target

# Extract time information from the datetime index
time_data = enhanced_data.index.get_level_values('time')

# Hour of day (cyclical encoding)
enhanced_data['hour'] = time_data.hour
enhanced_data['hour_sin'] = np.sin(2 * np.pi * enhanced_data['hour'] / 24)
enhanced_data['hour_cos'] = np.cos(2 * np.pi * enhanced_data['hour'] / 24)

# Day of year (cyclical encoding for seasonality)
enhanced_data['day_of_year'] = time_data.dayofyear
enhanced_data['season_sin'] = np.sin(2 * np.pi * enhanced_data['day_of_year'] / 365.25)
enhanced_data['season_cos'] = np.cos(2 * np.pi * enhanced_data['day_of_year'] / 365.25)

# Month and season indicators
enhanced_data['month'] = time_data.month
enhanced_data['is_winter'] = ((enhanced_data['month'] == 12) | (enhanced_data['month'] <= 2)).astype(int)
enhanced_data['is_spring'] = ((enhanced_data['month'] >= 3) & (enhanced_data['month'] <= 5)).astype(int) 
enhanced_data['is_summer'] = ((enhanced_data['month'] >= 6) & (enhanced_data['month'] <= 8)).astype(int)
enhanced_data['is_fall'] = ((enhanced_data['month'] >= 9) & (enhanced_data['month'] <= 11)).astype(int)

# Vapor Pressure and related calculations
def saturation_vapor_pressure(temp_celsius):
    """Calculate saturation vapor pressure using Magnus formula"""
    return 0.6112 * np.exp((17.67 * temp_celsius) / (temp_celsius + 243.5))

enhanced_data['sat_vapor_press'] = saturation_vapor_pressure(enhanced_data['temp'])
enhanced_data['actual_vapor_press'] = enhanced_data['sat_vapor_press'] * enhanced_data['rhum'] / 100
enhanced_data['vapor_pressure_deficit'] = enhanced_data['sat_vapor_press'] - enhanced_data['actual_vapor_press']

# Wet Bulb Temperature approximation
enhanced_data['wet_bulb_temp'] = enhanced_data['temp'] * np.arctan(0.151977 * (enhanced_data['rhum'] + 8.313659) ** 0.5) + np.arctan(enhanced_data['temp'] + enhanced_data['rhum']) - np.arctan(enhanced_data['rhum'] - 1.676331) + 0.00391838 * (enhanced_data['rhum'] ** 1.5) * np.arctan(0.023101 * enhanced_data['rhum']) - 4.686035

# Wind Chill (for temperatures below 10°C)
def wind_chill(temp, wind_speed):
    """Calculate wind chill factor"""
    mask = temp <= 10
    wc = np.full_like(temp, temp)  # Initialize with temperature
    wc[mask] = 13.12 + 0.6215 * temp[mask] - 11.37 * (wind_speed[mask] ** 0.16) + 0.3965 * temp[mask] * (wind_speed[mask] ** 0.16)
    return wc

enhanced_data['wind_chill'] = wind_chill(enhanced_data['temp'], enhanced_data['wspd'])

# Apparent Temperature (Heat Index for warm conditions)
def heat_index(temp, humidity):
    """Calculate heat index for temperatures above 26°C"""
    mask = temp >= 26
    hi = np.full_like(temp, temp)  # Initialize with temperature
    t = temp[mask]
    h = humidity[mask]
    hi[mask] = -42.379 + 2.04901523*t + 10.14333127*h - 0.22475541*t*h - 0.00683783*t*t - 0.05481717*h*h + 0.00122874*t*t*h + 0.00085282*t*h*h - 0.00000199*t*t*h*h
    return hi

enhanced_data['heat_index_advanced'] = heat_index(enhanced_data['temp'], enhanced_data['rhum'])

# Atmospheric stability indicators
enhanced_data['mixing_ratio'] = 0.622 * enhanced_data['actual_vapor_press'] / (enhanced_data['pres'] - enhanced_data['actual_vapor_press'])
enhanced_data['potential_temp'] = enhanced_data['temp'] * (1000 / enhanced_data['pres']) ** 0.286

def solar_elevation(lat, lon, datetime_utc):
    """Calculate solar elevation angle"""
    import math
    
    # Convert to numpy arrays if needed
    lat = np.array(lat)
    lon = np.array(lon) 
    
    # Day of year
    day_of_year = datetime_utc.dayofyear
    
    # Hour angle
    hour_angle = 15 * (datetime_utc.hour - 12) + lon
    
    # Solar declination
    declination = 23.45 * np.sin(np.radians(360 * (284 + day_of_year) / 365))
    
    # Solar elevation
    lat_rad = np.radians(lat)
    decl_rad = np.radians(declination)
    hour_rad = np.radians(hour_angle)
    
    elevation = np.arcsin(np.sin(lat_rad) * np.sin(decl_rad) + 
                         np.cos(lat_rad) * np.cos(decl_rad) * np.cos(hour_rad))
    
    return np.degrees(elevation)

# Calculate solar elevation for each data point
solar_elevations = []
for idx in enhanced_data.index:
    station_id, timestamp = idx
    lat = enhanced_data.loc[idx, 'latitude']
    lon = enhanced_data.loc[idx, 'longitude'] 
    elevation = solar_elevation(lat, lon, timestamp)
    solar_elevations.append(elevation)

enhanced_data['solar_elevation'] = solar_elevations
enhanced_data['is_daylight'] = (enhanced_data['solar_elevation'] > 0).astype(int)
enhanced_data['solar_intensity'] = np.maximum(0, np.sin(np.radians(enhanced_data['solar_elevation'])))

# Beaufort wind scale
def beaufort_scale(wind_speed):
    """Convert wind speed to Beaufort scale"""
    conditions = [
        wind_speed < 1,      # 0: Calm
        wind_speed < 5.5,    # 1: Light air  
        wind_speed < 11.9,   # 2: Light breeze
        wind_speed < 19.3,   # 3: Gentle breeze
        wind_speed < 28.7,   # 4: Moderate breeze
        wind_speed < 38.9,   # 5: Fresh breeze
        wind_speed < 49.9,   # 6: Strong breeze
        wind_speed < 61.8,   # 7: Near gale
        wind_speed < 74.6,   # 8: Gale
        wind_speed < 88.1,   # 9: Strong gale
        wind_speed < 102.4,  # 10: Storm
        wind_speed < 117.4,  # 11: Violent storm
    ]
    choices = list(range(12))
    return np.select(conditions, choices, default=12)  # 12: Hurricane

enhanced_data['beaufort_scale'] = beaufort_scale(enhanced_data['wspd'])
enhanced_data['wind_power'] = enhanced_data['wspd'] ** 3  # Wind power is proportional to cube of speed

wind_features = 2
print(f"   Added {wind_features} advanced wind features")

total_new_features = temporal_features + meteorological_features + solar_features + wind_features
final_features = original_features + total_new_features

# Store enhanced dataset for model training
data_with_additional_features = enhanced_data.copy()

IMPLEMENTING ADDITIONAL FEATURES
Step 1: Adding precipitation data...
Step 2: Adding temporal features...
Step 3: Adding meteorological calculations...
Step 4: Adding solar/astronomical features...

Starting with 147 features

🕐 Adding temporal features...

Starting with 147 features

🕐 Adding temporal features...
   Added 11 temporal features

🧮 Adding meteorological calculations...
   Added 11 temporal features

🧮 Adding meteorological calculations...
   Added 8 meteorological calculations

🌞 Adding solar/astronomical features...
   Added 8 meteorological calculations

🌞 Adding solar/astronomical features...
   Added 3 solar/astronomical features

💨 Adding advanced wind features...
   Added 2 advanced wind features

FEATURE ENHANCEMENT SUMMARY
Original features: 147
New features added: 24
Final feature count: 171
Total dataset shape: (408316, 173)

New feature categories:
🕐 Temporal: 11 features
🧮 Meteorological: 8 features
🌞 Solar: 3 features
💨 Wind: 2 features
   Added 3 solar/astr

In [15]:
# Prepare data for modeling
# Use enhanced dataset with all features
model_data = data_with_additional_features.copy()

# Split data by cities (not by rows)
train_data = model_data[model_data.index.get_level_values('station').isin(train_stations.index)]
test_data = model_data[model_data.index.get_level_values('station').isin(test_stations.index)]

# Separate features and target
X_train = train_data.drop('coco', axis=1)
y_train = train_data['coco']
X_test = test_data.drop('coco', axis=1)
y_test = test_data['coco']



Training set: 323123 samples with 172 features
Test set: 85193 samples
Target classes: coco
Clear     157456
Cloudy    137656
Rain       26035
Snow        1976
Name: count, dtype: int64


In [ ]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.utils.class_weight import compute_class_weight

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Set direct class weights (not multipliers)
direct_weights = {
    'Clear': 1.5,     # Direct weight
    'Cloudy': 3.0,   # Direct weight  
    'Rain': 20.0,     # Direct weight
    'Snow': 325.0    # Direct weight
}

# Apply direct weights
class_weight_dict = {}
for i, class_name in enumerate(label_encoder.classes_):
    class_weight_dict[i] = direct_weights[class_name]



model = XGBClassifier(
    n_estimators=1000, 
    learning_rate=0.1, 
    max_depth=6, 
    random_state=42, 
    n_jobs=1, 
    eval_metric='mlogloss'
)

# Convert class weights to sample weights for XGBoost
sample_weights = np.array([class_weight_dict[y] for y in y_train_encoded])

model.fit(X_train, y_train_encoded, sample_weight=sample_weights)

y_pred_encoded = model.predict(X_test)
y_pred = label_encoder.inverse_transform(y_pred_encoded)

accuracy = accuracy_score(y_test, y_pred)
print(f"XGBoost Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

Class distribution and direct weights:
  Clear: 157,456 samples | weight: 1.0
  Cloudy: 137,656 samples | weight: 1.25
  Rain: 26,035 samples | weight: 10.0
  Snow: 1,976 samples | weight: 500.0
XGBoost Accuracy: 0.6719 (67.19%)


In [28]:
xgb_importance = pd.Series(model.feature_importances_, index=X_train.columns)
xgb_importance = xgb_importance.sort_values(ascending=False)

# Create DataFrame for Altair
importance_df = pd.DataFrame({
    'feature': xgb_importance.head(15).index,
    'importance': xgb_importance.head(15).values
})

# Create Altair horizontal bar chart
importance_chart = alt.Chart(importance_df).mark_bar().encode(
    x=alt.X('importance:Q', title='Feature Importance'),
    y=alt.Y('feature:N', sort='-x', title='Feature'),
    color=alt.Color('importance:Q', scale=alt.Scale(scheme='blues'), legend=None),
    tooltip=['feature:N', alt.Tooltip('importance:Q', format='.4f')]
).properties(
    width=600,
    height=400,
    title='Top 15 Features - XGBoost Importance'
)

importance_chart

alt.Chart(...)

In [48]:
from sklearn.metrics import confusion_matrix

# Create confusion matrix with raw counts
cm = confusion_matrix(y_test, y_pred)

# Normalize by row to get percentages (what % of each true class is predicted as each class)
cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

# Create DataFrame for Altair heatmap
cm_data = []
for i, true_label in enumerate(label_encoder.classes_):
    for j, pred_label in enumerate(label_encoder.classes_):
        cm_data.append({
            'True Label': true_label,
            'Predicted Label': pred_label,
            'Percentage': cm_percent[i, j],
            'Text': f'{cm_percent[i, j]:.1f}%'
        })

cm_df = pd.DataFrame(cm_data)

# Create Altair heatmap
heatmap = alt.Chart(cm_df).mark_rect().encode(
    x=alt.X('Predicted Label:N', title='Predicted Label'),
    y=alt.Y('True Label:N', title='True Label'),
    color=alt.Color('Percentage:Q', 
                   scale=alt.Scale(scheme='blues', domain=[0, 100]),
                   title='Percentage (%)'),
    tooltip=['True Label:N', 'Predicted Label:N', alt.Tooltip('Percentage:Q', format='.1f')]
).properties(
    width=400,
    height=400,
    title=f'XGBoost Confusion Matrix (Row Percentages) - Accuracy: {accuracy:.2%}'
)

# Add text annotations
text = alt.Chart(cm_df).mark_text(
    align='center',
    baseline='middle',
    fontSize=12,
    fontWeight='bold'
).encode(
    x='Predicted Label:N',
    y='True Label:N',
    text='Text:N',
    color=alt.condition(alt.datum.Percentage > 50, alt.value('white'), alt.value('black'))
)

confusion_matrix_chart = heatmap + text
confusion_matrix_chart.show()



alt.LayerChart(...)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)



Accuracy: 0.4820 | Precision: 0.7016 | Recall: 0.4820 | F1: 0.4181
              precision    recall  f1-score   support

       Clear       0.92      0.18      0.30     44713
      Cloudy       0.42      0.94      0.58     32634
        Rain       0.66      0.29      0.41      6464
        Snow       0.48      0.39      0.43      1382

    accuracy                           0.48     85193
   macro avg       0.62      0.45      0.43     85193
weighted avg       0.70      0.48      0.42     85193

              precision    recall  f1-score   support

       Clear       0.92      0.18      0.30     44713
      Cloudy       0.42      0.94      0.58     32634
        Rain       0.66      0.29      0.41      6464
        Snow       0.48      0.39      0.43      1382

    accuracy                           0.48     85193
   macro avg       0.62      0.45      0.43     85193
weighted avg       0.70      0.48      0.42     85193



In [ ]:
from sklearn.metrics import accuracy_score

test_data_with_stations = test_data.reset_index()
test_predictions = model.predict(X_test)
test_predictions = label_encoder.inverse_transform(test_predictions)

city_performance = []
test_station_names = test_stations.index.tolist()

for station_id in test_station_names:
    station_mask = test_data_with_stations['station'] == station_id
    station_indices = test_data_with_stations[station_mask].index
    valid_indices = [i for i in station_indices if i < len(y_test)]
    
    if len(valid_indices) > 10:
        station_y_true = y_test.iloc[valid_indices]
        station_y_pred = test_predictions[valid_indices]
        city_accuracy = accuracy_score(station_y_true, station_y_pred)
        
        city_performance.append({
            'station_id': station_id,
            'name': test_stations.loc[station_id, 'name'],
            'longitude': test_stations.loc[station_id, 'longitude'],
            'latitude': test_stations.loc[station_id, 'latitude'],
            'accuracy': city_accuracy,
            'samples': len(valid_indices)
        })

city_perf_df = pd.DataFrame(city_performance)

                                            name  accuracy  samples
0  San Diego / Town And Country Mobile Home Park     0.702     8496
1                      Boonville / Windsor Place     0.670     8494
2                                      Gila Bend     0.782     8495
3                                        Mc Call     0.603     8492
4                                          Dixon     0.599     8496
5            Indianapolis International  Airport     0.695     8738
6            Southland Field-Sulphur / Moss Lake     0.653     8499
7                                     Ponca City     0.710     8495
8                    Rockingham / Highland Pines     0.680     8492
9                                   Lamar / Beta     0.694     8496


In [21]:
us_states = alt.topo_feature(datasets.us_10m.url, 'states')
base_map = (
    alt.Chart(us_states)
    .mark_geoshape(fill="#f0f0f0", stroke="black")
    .project("albersUsa")
)

train_points = (
    alt.Chart(train_stations.reset_index())
    .mark_circle(size=100, opacity=0.6)
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        color=alt.value("gray"),
        tooltip=["name:N", "id:N"]
    )
)

test_points = (
    alt.Chart(city_perf_df)
    .mark_circle(size=300, opacity=0.9, stroke="black", strokeWidth=2)
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        color=alt.Color("accuracy:Q", 
                       scale=alt.Scale(scheme="viridis", domain=[0.4, 1.0]),
                       title="XGBoost Accuracy"),
        tooltip=[
            alt.Tooltip("name:N", title="City"),
            alt.Tooltip("accuracy:Q", title="XGBoost Accuracy", format=".3f"),
            alt.Tooltip("samples:Q", title="Test Samples"),
            alt.Tooltip("longitude:Q", title="Longitude", format=".2f"),
            alt.Tooltip("latitude:Q", title="Latitude", format=".2f")
        ]
    )
)

final_map = (base_map + train_points + test_points).properties(
    width=800,
    height=500,
    title="XGBoost Model Performance: 40 Training Cities (Gray) + 10 Test Cities (Colored by Accuracy)"
).resolve_scale(
    color='independent'
)

final_map

alt.LayerChart(...)